## Q1. Running Elastic

In [32]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()

## What's the version.build_hash value?

In [2]:
es_client.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

## Getting the Data

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [49]:
# documents

## Q2 Indexing the data

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/WgHbbEfaSiCt5rL5zbUgUw] already exists')

In [ ]:
for doc in tqdm(documents): 
    es_client.index(index=index_name, document=doc)

## Q3. Searching

In [ ]:
query = 'How do execute a command on a Kubernetes pod?'

In [47]:
##def elastic_search(query): 
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }         
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

## What's the score for the top ranking result?

In [48]:
response['hits']['hits'][0]['_score']

73.38676

## Q4. Filtering

In [8]:
query = 'How do copy a file to a Docker container?'

In [9]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [10]:
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [11]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [12]:
context = ""
for doc in result_docs:
    context += context_template.format(question=doc["question"], text=doc["text"]) + "\n\n"


context = context.strip()
print(context)  # <-- This will show the actual filled-in result

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [13]:
query_2 = 'How do I execute a command in a running docker container?'

In [14]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [15]:
prompt = prompt_template.format(question=query_2, context=context).strip()

In [16]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [17]:
print(len(prompt))

1462


## Q6. Tokens

In [18]:
pip install tiktoken


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import tiktoken

In [23]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [29]:
print(len(encoding.encode(prompt)))

322


## Bonus: generating the answer (ungraded)

In [33]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
    )

In [46]:
search_results = result_docs
prompt = prompt
answer = response

answer.choices[0].message.content

'To execute a command in a running Docker container, you first need to find the container ID of the running container using the `docker ps` command. Once you have the container ID, you can use the `docker exec` command to execute a command inside the container. The basic syntax is:\n\n```bash\ndocker exec -it <container-id> <command>\n```\n\nFor example, to open a bash shell in the container, you would use:\n\n```bash\ndocker exec -it <container-id> bash\n```'